In [5]:
from os import listdir
from fragment_file_reader import fragment_file_reader
from datetime import datetime
from find_closest_fragment import find_closest_fragment

# What directory are the text files stored in?
basedir = '/Users/fraser/Github/MagneticFragmentation/fragment_properties/'

# Get list of files to use
files = [basedir + i for i in listdir(basedir)]

# Split the list into two lists, one for positive fragments and one for negative fragments
neg_files = [s for s in files if 'n.' in s]
pos_files = [s for s in files if 'p.' in s]

# Set up the individual store dictionaries
neg_store = {}
neg_store_number = 0
pos_store = {}
pos_store_number = 0

# For the first image, there is no previous timestep to compare against. All fragments are put into the store.
[neg_dates, neg_latitudes, neg_longitudes] = fragment_file_reader(neg_files[0])

for i in range(len(neg_dates)):
    neg_store[str(neg_store_number)] = [neg_dates[i]], [neg_latitudes[i]], [neg_longitudes[i]]
    neg_store_number += 1

[pos_dates, pos_latitudes, pos_longitudes] = fragment_file_reader(pos_files[0])

for i in range(len(pos_dates)):
    pos_store[str(pos_store_number)] = [pos_dates[i]], [pos_latitudes[i]], [pos_longitudes[i]]
    pos_store_number += 1

# For each subsequent image (using the negative file number to keep track, but the positive one is the same)
for image in range(1, len(neg_files)):
    # Get fragments from new image
    [neg_dates_new, neg_latitudes_new, neg_longitudes_new] = fragment_file_reader(neg_files[image])
    # Get timestamp of old image to use to search the fragment store
    neg_date_old = fragment_file_reader(neg_files[image-1])[0][0]
    # Get fragments from new image
    [pos_dates_new, pos_latitudes_new, pos_longitudes_new] = fragment_file_reader(pos_files[image])
    # Get timestamp of old image to use to search the fragment store
    pos_date_old = fragment_file_reader(pos_files[image-1])[0][0]
    
    # Search the region store and pull out all fragments that are in the old image
    # Reset the negative key list and old lat long lists
    neg_key_list = []
    neg_latitudes_old = []
    neg_longitudes_old = []
    for key in neg_store:
        if neg_store[key][0][-1] == neg_date_old:
            # Find which keys in the dictionary contain fragments from the previous image and get their key, lat, and long
            neg_key_list.append(key)
            neg_latitudes_old.append(neg_store[key][1][-1])
            neg_longitudes_old.append(neg_store[key][2][-1])
    
    # Reset the positive key list and old lat long lists
    pos_key_list = []
    pos_latitudes_old = []
    pos_longitudes_old = []
    for key in pos_store:
        if pos_store[key][0][-1] == pos_date_old:
            # Find which keys in the dictionary contain fragments from the previous image and get their key, lat, and long
            pos_key_list.append(key)
            pos_latitudes_old.append(pos_store[key][1][-1])
            pos_longitudes_old.append(pos_store[key][2][-1])
    
    # Calculate the time difference between the old and new images to remove solar rotation
    time_delta = (neg_dates_new[0] - neg_date_old).seconds/86400
    # For each fragment in the new image, find the closest one in space in the old image
    for fragment in range(len(neg_latitudes_new)):
        print(neg_latitudes_old)
        [distance, index] = find_closest_fragment(neg_latitudes_new[fragment], neg_longitudes_new[fragment], neg_latitudes_old, neg_longitudes_old, time_delta)
        
        # Set a maximum reasonable distance based on the timedelta
        # If fragment is close enough, append it to an old entry
        # Otherwise, create a new one.
        if (distance / time_delta) < 5:
        
            # Then append that fragments properties to the key of the old one
            append_key = neg_key_list[index]
            neg_store[append_key][0].append(neg_dates_new[fragment])
            neg_store[append_key][1].append(neg_latitudes_new[fragment])
            neg_store[append_key][2].append(neg_longitudes_new[fragment])

            # And remove that fragment from the 'old' list as only one new can link to one old
            if len(neg_latitudes_old) > 1:
                del neg_key_list[index]
                del neg_latitudes_old[index]
                del neg_longitudes_old[index]
            else:
                break
        
        else:
            neg_store[str(neg_store_number)] = [neg_dates_new[fragment]], [neg_latitudes_new[fragment]], [neg_longitudes_new[fragment]]
            neg_store_number += 1

    # Calculate the time difference between the old and new images to remove solar rotation
    time_delta = (pos_dates_new[0] - pos_date_old).seconds/86400
    # For each fragment in the new image, find the closest one in space in the old image
    for fragment in range(len(pos_latitudes_new)):
        [distance, index] = find_closest_fragment(pos_latitudes_new[fragment], pos_longitudes_new[fragment], pos_latitudes_old, pos_longitudes_old, time_delta)
        
        # Set a maximum reasonable distance based on the timedelta
        # If fragment is close enough, append it to an old entry
        # Otherwise, create a new one.
        if (distance / time_delta) < 5:
        
            # Then append that fragments properties to the key of the old one
            append_key = pos_key_list[index]
            pos_store[append_key][0].append(pos_dates_new[fragment])
            pos_store[append_key][1].append(pos_latitudes_new[fragment])
            pos_store[append_key][2].append(pos_longitudes_new[fragment])

            # And remove that fragment from the 'old' list as only one new can link to one old
            if len(pos_latitudes_old) > 1:
                del pos_key_list[index]
                del pos_latitudes_old[index]
                del pos_longitudes_old[index]
            else:
                break
        
        else:
            pos_store[str(pos_store_number)] = [pos_dates_new[fragment]], [pos_latitudes_new[fragment]], [pos_longitudes_new[fragment]]
            pos_store_number += 1


[-6.5, -10.9, -5.3, -5.8, -4.1, -3.7, -11.6, -6.7, -6.0, -7.8, -5.7, -6.6, -13.7, -8.2, -5.5, -5.9, -7.6, -8.2, -13.5, -8.1, -13.4, -11.3, -13.4, -3.0, -6.4, -13.1, -5.9, -9.0, -5.7, -7.5, -14.5, -16.5, -6.2, -4.9, -8.3, -5.0, -6.4, -8.8, -8.6, -6.8, -16.9, -6.2, -8.1, -6.5, -2.9, -5.4, -5.6, -6.2, -5.8, -6.2, -5.1, -7.4, -16.6, -7.7, -4.9, -6.0, -3.6, -4.4, -13.1, -4.9, -4.9, -8.0, -6.2, -4.5, -6.4, -11.3, -6.5, -6.9, -4.3, -7.9, -5.2, -6.3, -11.9, -6.7, -4.9, -6.7, -4.9, -6.3, -13.3, -6.4, -4.5, -11.3, -9.1, -13.8, -6.6, -13.6, -6.3, -6.1, -16.3, -5.1, -5.0, -6.6, -13.8, -5.1, -6.2, -13.2, -8.8, -5.9, -8.7]
[-6.5, -10.9, -5.3, -5.8, -4.1, -3.7, -11.6, -6.7, -6.0, -7.8, -5.7, -6.6, -13.7, -8.2, -5.5, -5.9, -7.6, -8.2, -13.5, -8.1, -13.4, -11.3, -13.4, -3.0, -6.4, -13.1, -5.9, -9.0, -5.7, -7.5, -14.5, -16.5, -6.2, -4.9, -8.3, -5.0, -6.4, -8.8, -8.6, -6.8, -16.9, -6.2, -8.1, -6.5, -5.4, -5.6, -6.2, -5.8, -6.2, -5.1, -7.4, -16.6, -7.7, -4.9, -6.0, -3.6, -4.4, -13.1, -4.9, -4.9, -8.0, -6.

[-6.5, -3.6, -10.9, -5.3, -5.8, -4.9, -4.1, -5.0, -7.7, -6.7, -8.1, -6.0, -7.8, -5.7, -6.6, -13.7, -8.2, -5.1, -11.3, -8.8, -13.4, -5.5, -7.6, -13.5, -8.1, -13.7, -13.4, -11.3, -13.4, -3.0, -6.0, -5.1, -13.9, -6.4, -7.8, -9.0, -7.5, -14.5, -7.6, -7.8, -16.5, -6.2, -13.8, -10.8, -4.9, -13.6, -4.9, -6.4, -6.4, -8.6, -16.9, -6.2, -6.5, -2.9, -5.4, -5.6, -4.2, -7.6, -6.2, -5.8, -5.9, -5.1, -7.4, -6.4, -8.0, -3.6, -4.4, -16.5, -13.1, -4.9, -8.0, -6.2, -4.5, -6.4, -11.3, -6.5, -5.7, -6.9, -5.1, -4.3, -7.9, -5.2, -11.9, -6.7, -4.9, -3.7, -6.7, -6.4, -4.5, -9.1, -6.6, -6.3, -6.3, -6.1, -16.3, -10.9, -6.6, -13.8, -6.2, -12.8, -13.2, -8.8, -5.9, -8.7]
[-6.5, -3.6, -10.9, -5.3, -5.8, -4.9, -4.1, -5.0, -7.7, -6.7, -8.1, -6.0, -7.8, -5.7, -6.6, -13.7, -8.2, -5.1, -11.3, -8.8, -13.4, -5.5, -7.6, -13.5, -8.1, -13.7, -13.4, -11.3, -13.4, -3.0, -6.0, -5.1, -13.9, -6.4, -7.8, -9.0, -7.5, -14.5, -7.6, -7.8, -16.5, -6.2, -13.8, -10.8, -4.9, -13.6, -4.9, -6.4, -6.4, -8.6, -16.9, -6.2, -6.5, -2.9, -5.4, -5.

[-4.3, -6.5, -3.6, -6.4, -10.9, -3.8, -5.3, -4.9, -4.1, -4.9, -5.0, -6.4, -8.1, -6.0, -7.8, -6.6, -13.7, -8.2, -8.8, -14.6, -13.4, -16.3, -5.5, -7.6, -13.5, -8.0, -11.2, -6.0, -5.1, -13.9, -6.4, -7.8, -9.0, -7.5, -6.2, -8.1, -11.3, -4.9, -13.6, -6.7, -6.5, -10.8, -16.9, -6.2, -6.5, -16.5, -5.4, -5.2, -5.6, -6.3, -13.2, -6.5, -5.1, -6.2, -13.8, -5.9, -5.1, -7.4, -6.2, -4.4, -5.7, -16.5, -5.7, -8.2, -5.6, -6.2, -4.5, -2.7, -11.3, -6.5, -5.7, -6.9, -7.9, -5.2, -11.9, -8.1, -4.9, -6.7, -4.3, -9.1, -6.6, -6.3, -10.9, -6.6, -13.8, -6.2, -13.2, -8.8, -5.9, -8.6, -8.7, -6.3]
[-4.3, -6.5, -3.6, -6.4, -10.9, -3.8, -5.3, -4.9, -4.1, -4.9, -5.0, -6.4, -8.1, -6.0, -7.8, -6.6, -13.7, -8.2, -8.8, -14.6, -13.4, -16.3, -5.5, -7.6, -13.5, -8.0, -11.2, -6.0, -5.1, -13.9, -6.4, -7.8, -9.0, -7.5, -6.2, -8.1, -11.3, -4.9, -13.6, -6.7, -6.5, -10.8, -16.9, -6.2, -6.5, -16.5, -5.4, -5.2, -5.6, -6.3, -13.2, -6.5, -5.1, -6.2, -13.8, -5.9, -5.1, -7.4, -6.2, -4.4, -5.7, -16.5, -5.7, -8.2, -5.6, -6.2, -4.5, -11.3, 

[-4.3, -5.3, -6.6, -13.7, -8.2, -14.6, -13.4, -16.3, -7.6, -13.5, -8.0, -11.2, -6.0, -5.1, -13.9, -6.4, -7.5, -11.3, -13.6, -16.9, -16.5, -5.2, -6.3, -13.2, -6.2, -13.8, -4.4, -5.7, -16.5, -5.7, -11.3, -6.5, -11.9, -8.1, -4.3, -10.9, -13.8, -13.2, -5.9]
[-4.3, -5.3, -6.6, -13.7, -8.2, -14.6, -13.4, -16.3, -7.6, -13.5, -8.0, -11.2, -6.0, -5.1, -13.9, -6.4, -7.5, -11.3, -13.6, -16.9, -16.5, -5.2, -6.3, -13.2, -6.2, -13.8, -4.4, -5.7, -16.5, -5.7, -6.5, -11.9, -8.1, -4.3, -10.9, -13.8, -13.2, -5.9]
[-4.3, -5.3, -6.6, -13.7, -8.2, -14.6, -13.4, -16.3, -7.6, -13.5, -8.0, -6.0, -5.1, -13.9, -6.4, -7.5, -11.3, -13.6, -16.9, -16.5, -5.2, -6.3, -13.2, -6.2, -13.8, -4.4, -5.7, -16.5, -5.7, -6.5, -11.9, -8.1, -4.3, -10.9, -13.8, -13.2, -5.9]
[-4.3, -5.3, -6.6, -13.7, -8.2, -14.6, -13.4, -16.3, -7.6, -13.5, -8.0, -6.0, -5.1, -13.9, -6.4, -7.5, -13.6, -16.9, -16.5, -5.2, -6.3, -13.2, -6.2, -13.8, -4.4, -5.7, -16.5, -5.7, -6.5, -11.9, -8.1, -4.3, -10.9, -13.8, -13.2, -5.9]
[-4.3, -5.3, -6.6, -13.7, 

[-6.4, -5.8, -13.6, -11.2, -9.0, -6.6, -4.3, -16.5, -7.4, -16.9, -13.2, -5.8, -10.8, -6.7, -13.8, -13.8, -13.2, -8.8, -5.0, -6.0, -13.9, -6.1, -13.4, -14.6, -13.5, -16.3, -11.6, -11.2, -6.5, -5.4, -5.1, -6.2, -3.7, -11.3, -11.3, -8.0, -6.3, -6.6, -11.9, -6.6]
[-6.4, -5.8, -13.6, -11.2, -9.0, -6.6, -4.3, -16.5, -7.4, -16.9, -13.2, -5.8, -10.8, -6.7, -13.8, -13.8, -13.2, -8.8, -5.0, -6.0, -13.9, -6.1, -13.4, -14.6, -13.5, -16.3, -11.6, -6.5, -5.4, -5.1, -6.2, -3.7, -11.3, -11.3, -8.0, -6.3, -6.6, -11.9, -6.6]
[-6.4, -5.8, -13.6, -11.2, -9.0, -6.6, -4.3, -16.5, -7.4, -16.9, -13.2, -5.8, -10.8, -6.7, -13.8, -13.8, -13.2, -8.8, -5.0, -6.0, -13.9, -6.1, -13.4, -14.6, -13.5, -16.3, -11.6, -6.5, -5.4, -5.1, -6.2, -3.7, -11.3, -11.3, -8.0, -6.3, -6.6, -11.9, -6.6]
[-6.4, -5.8, -13.6, -9.0, -6.6, -4.3, -16.5, -7.4, -16.9, -13.2, -5.8, -10.8, -6.7, -13.8, -13.8, -13.2, -8.8, -5.0, -6.0, -13.9, -6.1, -13.4, -14.6, -13.5, -16.3, -11.6, -6.5, -5.4, -5.1, -6.2, -3.7, -11.3, -11.3, -8.0, -6.3, -6.6, -

...

[-7.3, -4.9, -7.6, -13.2, -6.6, -6.2, -6.3, -6.1, -6.3, -5.5, -5.6, -6.4, -6.3, -6.2, -6.2, -6.1, -8.0, -6.2, -7.8, -13.6, -5.1, -10.9, -2.7, -7.7, -6.7, -8.8, -9.1, -16.6, -4.3, -5.9, -6.2, -4.1, -13.6, -8.2, -8.5, -6.6, -8.1, -3.9, -6.2, -6.0, -3.5, -4.2, -4.4, -5.9, -5.0, -6.0, -11.2, -6.1, -6.5, -8.1, -3.6, -7.9, -4.9, -8.7, -2.7, -7.6, -5.8, -6.4, -6.2, -6.4, -13.2, -6.4, -10.9, -6.0, -5.6, -6.3, -5.6, -6.9, -4.9, -6.0, -13.1, -6.5, -8.2, -13.3, -11.2, -11.1, -6.2, -6.5, -6.6, -6.2, -5.1, -13.2, -4.4, -8.0, -8.1, -5.7, -5.7, -8.1, -11.9, -13.9, -6.1, -4.9, -13.6, -6.5, -8.6, -8.5, -13.9, -6.7, -5.5, -6.3, -5.3, -11.8, -8.7, -5.7]
[-7.3, -4.9, -7.6, -13.2, -6.6, -6.2, -6.3, -6.1, -6.3, -5.5, -5.6, -6.4, -6.3, -6.2, -6.2, -6.1, -8.0, -6.2, -7.8, -13.6, -5.1, -10.9, -2.7, -7.7, -6.7, -8.8, -9.1, -16.6, -4.3, -5.9, -6.2, -4.1, -13.6, -8.2, -8.5, -6.6, -8.1, -3.9, -6.2, -6.0, -3.5, -4.2, -4.4, -5.9, -5.0, -6.0, -11.2, -6.1, -6.5, -8.1, -3.6, -7.9, -4.9, -8.7, -7.6, -5.8, -6.4, -6.2, -6

[-13.4, -6.3, -6.6, -6.4, -6.2, -6.3, -5.1, -6.1, -6.0, -5.6, -7.9, -6.4, -5.5, -8.5, -6.3, -8.8, -6.1, -8.0, -6.2, -8.2, -7.8, -13.6, -5.1, -10.9, -4.9, -5.3, -7.7, -6.7, -8.8, -9.1, -4.3, -5.9, -6.2, -4.3, -4.1, -8.3, -7.5, -6.2, -6.0, -5.2, -5.9, -11.2, -6.1, -6.5, -8.1, -5.7, -6.1, -8.7, -2.7, -7.6, -6.4, -6.2, -14.6, -6.4, -6.7, -10.9, -6.0, -5.8, -5.6, -6.3, -5.6, -5.0, -6.9, -4.9, -6.0, -13.3, -16.6, -13.1, -6.5, -8.2, -13.2, -11.2, -5.9, -6.2, -3.5, -6.5, -4.9, -6.6, -6.2, -6.6, -5.1, -13.2, -4.4, -8.0, -8.1, -11.8, -7.4, -8.1, -11.9, -13.9, -6.1, -4.9, -13.6, -6.5, -8.5, -13.9, -6.7, -5.3, -6.0, -3.6, -6.8, -11.8, -5.0, -8.7, -5.7, -6.3]
[-13.4, -6.3, -6.6, -6.4, -6.2, -6.3, -5.1, -6.1, -6.0, -5.6, -7.9, -6.4, -5.5, -8.5, -6.3, -8.8, -6.1, -8.0, -6.2, -8.2, -7.8, -13.6, -5.1, -10.9, -4.9, -5.3, -7.7, -6.7, -8.8, -9.1, -4.3, -5.9, -6.2, -4.3, -4.1, -8.3, -7.5, -6.2, -6.0, -5.2, -5.9, -11.2, -6.1, -6.5, -8.1, -5.7, -6.1, -8.7, -7.6, -6.4, -6.2, -14.6, -6.4, -6.7, -10.9, -6.0, -5

[-5.5, -6.3, -6.4, -6.7, -6.2, -14.8, -6.3, -5.0, -5.1, -11.3, -8.7, -6.0, -7.9, -8.5, -6.3, -5.6, -8.0, -7.6, -6.2, -6.4, -8.1, -7.8, -13.6, -5.1, -5.5, -4.9, -7.7, -6.7, -8.8, -8.3, -9.1, -4.2, -8.1, -6.5, -5.9, -6.2, -4.3, -4.1, -5.9, -8.3, -6.2, -6.2, -6.5, -5.2, -5.9, -10.9, -5.6, -11.2, -6.5, -8.1, -6.1, -2.7, -6.2, -6.4, -10.9, -5.8, -6.3, -5.0, -14.5, -6.9, -5.6, -4.9, -6.0, -4.5, -13.4, -16.6, -13.1, -4.9, -8.2, -8.7, -11.2, -3.5, -5.9, -6.5, -4.9, -6.6, -6.2, -6.6, -5.1, -13.2, -4.4, -8.0, -8.1, -7.4, -11.9, -13.9, -6.1, -4.9, -13.6, -6.5, -13.9, -6.7, -8.9, -5.3, -7.6, -6.0, -3.6, -8.6, -7.9, -6.8, -5.7, -11.8, -5.0, -13.3, -8.7, -11.8, -6.1, -5.7, -6.3]
[-5.5, -6.3, -6.4, -6.7, -6.2, -14.8, -6.3, -5.0, -5.1, -11.3, -8.7, -6.0, -7.9, -8.5, -6.3, -5.6, -8.0, -7.6, -6.2, -6.4, -8.1, -7.8, -13.6, -5.1, -5.5, -4.9, -7.7, -6.7, -8.8, -8.3, -9.1, -4.2, -8.1, -6.5, -5.9, -6.2, -4.3, -4.1, -5.9, -8.3, -6.2, -6.2, -6.5, -5.2, -5.9, -10.9, -5.6, -11.2, -6.5, -8.1, -6.1, -6.2, -6.4, -1

[-5.5, -6.9, -6.3, -6.4, -6.7, -6.2, -13.3, -14.8, -6.3, -5.0, -11.3, -8.7, -7.9, -8.5, -6.3, -5.6, -6.1, -8.0, -7.6, -6.2, -6.4, -8.1, -7.8, -13.6, -5.1, -5.8, -4.9, -7.7, -8.8, -8.3, -9.1, -6.5, -5.9, -4.3, -6.1, -4.1, -8.6, -5.9, -13.4, -6.2, -5.7, -6.5, -4.3, -6.8, -8.2, -6.5, -8.1, -5.1, -2.7, -6.4, -10.9, -13.2, -5.8, -6.3, -8.1, -14.5, -6.0, -13.9, -13.4, -16.6, -11.2, -6.2, -5.9, -6.5, -10.9, -4.9, -6.6, -6.2, -6.6, -5.1, -10.9, -13.2, -4.4, -8.0, -7.4, -13.9, -4.9, -5.0, -6.7, -6.0, -8.9, -5.3, -14.6, -7.6, -6.0, -3.6, -8.6, -7.9, -11.8, -5.0, -8.4, -8.7, -11.8, -6.1, -5.7, -13.6, -6.3]
[-5.5, -6.9, -6.3, -6.4, -6.7, -6.2, -13.3, -14.8, -6.3, -5.0, -11.3, -8.7, -7.9, -8.5, -6.3, -5.6, -6.1, -8.0, -7.6, -6.2, -6.4, -8.1, -7.8, -13.6, -5.1, -5.8, -4.9, -7.7, -8.8, -8.3, -9.1, -6.5, -5.9, -4.3, -6.1, -4.1, -8.6, -5.9, -13.4, -6.2, -5.7, -6.5, -4.3, -6.8, -8.2, -6.5, -8.1, -5.1, -6.4, -10.9, -13.2, -5.8, -6.3, -8.1, -14.5, -6.0, -13.9, -13.4, -16.6, -11.2, -6.2, -5.9, -6.5, -10.9,

[-6.9, -8.5, -4.5, -6.4, -6.7, -6.2, -13.3, -14.8, -6.3, -5.0, -11.3, -6.0, -6.4, -7.9, -6.3, -8.5, -6.3, -3.5, -11.0, -6.1, -8.0, -6.2, -6.1, -5.6, -6.4, -8.1, -7.8, -5.1, -8.1, -11.9, -7.6, -5.8, -4.9, -8.8, -8.3, -9.1, -6.6, -6.5, -5.9, -4.1, -8.6, -4.9, -6.7, -6.2, -5.7, -8.0, -13.2, -6.8, -6.5, -8.2, -6.7, -6.5, -5.1, -2.7, -6.4, -14.5, -10.9, -8.2, -13.2, -5.8, -6.3, -13.3, -6.0, -13.9, -7.7, -16.6, -5.7, -6.2, -8.1, -6.2, -5.9, -6.8, -5.3, -6.5, -10.9, -6.2, -13.5, -6.6, -5.6, -5.1, -4.4, -8.0, -11.2, -7.9, -13.9, -4.9, -5.0, -6.0, -8.9, -7.6, -6.0, -3.6, -7.9, -11.8, -11.8, -8.0, -13.6, -6.3, -4.4]
[-6.9, -8.5, -4.5, -6.4, -6.7, -6.2, -13.3, -14.8, -6.3, -5.0, -11.3, -6.0, -6.4, -7.9, -6.3, -8.5, -6.3, -3.5, -11.0, -6.1, -8.0, -6.2, -6.1, -5.6, -6.4, -8.1, -7.8, -5.1, -8.1, -11.9, -7.6, -5.8, -4.9, -8.8, -8.3, -9.1, -6.6, -6.5, -5.9, -4.1, -8.6, -4.9, -6.7, -6.2, -5.7, -8.0, -13.2, -6.8, -6.5, -8.2, -6.7, -6.5, -5.1, -6.4, -14.5, -10.9, -8.2, -13.2, -5.8, -6.3, -13.3, -6.0, -13

In [6]:
neg_store

{'0': ([datetime.datetime(2016, 7, 7, 23, 58, 47, 300000),
   datetime.datetime(2016, 7, 8, 0, 10, 47, 300000)],
  [-2.9, -2.9],
  [-36.1, -36.0]),
 '1': ([datetime.datetime(2016, 7, 7, 23, 58, 47, 300000),
   datetime.datetime(2016, 7, 8, 0, 10, 47, 300000)],
  [-3.0, -3.0],
  [-28.6, -28.5]),
 '10': ([datetime.datetime(2016, 7, 7, 23, 58, 47, 300000)], [-4.9], [-33.1]),
 '100': ([datetime.datetime(2016, 7, 8, 0, 10, 47, 300000)], [-3.7], [-32.1]),
 '101': ([datetime.datetime(2016, 7, 8, 0, 10, 47, 300000)], [-4.2], [-31.7]),
 '102': ([datetime.datetime(2016, 7, 8, 0, 10, 47, 300000)], [-4.9], [-33.1]),
 '103': ([datetime.datetime(2016, 7, 8, 0, 10, 47, 300000)], [-4.9], [-32.8]),
 '104': ([datetime.datetime(2016, 7, 8, 0, 10, 47, 300000),
   datetime.datetime(2016, 7, 8, 0, 22, 47, 300000),
   datetime.datetime(2016, 7, 8, 0, 34, 47, 300000),
   datetime.datetime(2016, 7, 8, 0, 46, 47, 300000)],
  [-4.9, -4.9, -4.9, -4.9],
  [-32.8, -32.7, -32.6, -32.5]),
 '105': ([datetime.datetime(